## Library Import

In [62]:
import pandas as pd
from matplotlib import pyplot
from pandas.plotting import lag_plot
from IPython.display import display,Markdown
import plotly
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.simplefilter('ignore')
import datetime as dt

import fbprophet
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.plot import plot_forecast_component
from prophet_plots_plotly import plot_model_components, plot_model

buttons_to_remove = ['zoom2d','zoomIn2d','zoomOut2d','toImage',
                     'sendDataToCloud',
                     'editInChartStudio',
                     'select2d',
                     'lasso2d',]

scatter_config = {'displayModeBar': 'Always',
                 "displaylogo": False,
                 'modeBarButtons': 'toggleHover ',
                 'modeBarButtonsToRemove': buttons_to_remove,}


## Import & Modify CPI Data

In [63]:
'''
Raw CPI Data
'''

cpi_raw = pd.read_excel('CPIRawData.xlsx',engine='openpyxl',sheet_name = 'BLS Data Series',skiprows=11)

'''
Import CPI data, remove monthly data, calculate inflation, and find rolling avg of CPI and inflation
'''

cpi_yearly = cpi_raw.copy()

cpi_yearly = cpi_yearly[['Year','Annual']]
cpi_yearly.rename(columns={"Annual": "CPI",'Year': 'Date'},inplace=True)

window = 3

cpi_yearly['RollingAvgCPI'] = cpi_yearly.CPI.rolling(window).mean()
cpi_yearly['RollingAvgCPI_low'] = cpi_yearly.CPI.rolling(window).min()
cpi_yearly['RollingAvgCPI_high'] = cpi_yearly.CPI.rolling(window).max()

cpi_yearly['Inflation'] = (cpi_yearly.CPI.diff()/ cpi_yearly['CPI'].shift(1))*100
cpi_yearly['RollingAvgInflation'] = cpi_yearly.Inflation.rolling(window).mean()
cpi_yearly['RollingAvgInflation_low'] = cpi_yearly.Inflation.rolling(window).min()
cpi_yearly['RollingAvgInflation_high'] = cpi_yearly.Inflation.rolling(window).max()

display(Markdown('<h2>Yearly CPI and Inflation Data</h2>'))
display(cpi_yearly.head())


'''
Transform CPI data into monthly format and calculate inflation
'''
cpi_monthly =  cpi_raw.copy()
#drop unnecessary columns
cpi_monthly.drop(columns = ['Annual','HALF1','HALF2'], inplace = True)

#use pd.melt to combine columns and rows
cpi_monthly = pd.melt(cpi_monthly, id_vars=["Year"], var_name="Month", value_name = "CPI")
#modify columns to get usable format and calculate inflation
cpi_monthly['Date'] = cpi_monthly.Month + " " + cpi_monthly.Year.map(str)
cpi_monthly = cpi_monthly[['Date','CPI']]
cpi_monthly["Date"] = pd.to_datetime(cpi_monthly.Date, format="%b %Y", dayfirst=True)
cpi_monthly = cpi_monthly.sort_values("Date")
cpi_monthly ['Date'] = cpi_monthly['Date'].dt.strftime('%m/%d/%Y')
cpi_monthly['Inflation'] = (cpi_monthly.CPI.diff()/ cpi_monthly['CPI'].shift(1))*100

cpi_monthly['RollingAvgCPI'] = cpi_monthly.CPI.rolling(window).mean()
cpi_monthly['RollingAvgCPI_low'] = cpi_monthly.CPI.rolling(window).min()
cpi_monthly['RollingAvgCPI_high'] = cpi_monthly.CPI.rolling(window).max()

cpi_monthly['Inflation'] = (cpi_monthly.CPI.diff()/ cpi_monthly['CPI'].shift(1))*100
cpi_monthly['RollingAvgInflation'] = cpi_monthly.Inflation.rolling(window).mean()
cpi_monthly['RollingAvgInflation_low'] = cpi_monthly.Inflation.rolling(window).min()
cpi_monthly['RollingAvgInflation_high'] = cpi_monthly.Inflation.rolling(window).max()

display(Markdown('<h2>Monthly CPI and Inflation Data</h2>'))
display(cpi_monthly.head())

<h2>Yearly CPI and Inflation Data</h2>

,Date,CPI,RollingAvgCPI,RollingAvgCPI_low,RollingAvgCPI_high,Inflation,RollingAvgInflation,RollingAvgInflation_low,RollingAvgInflation_high
0,1913,9.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1914,10.0,NaN,NaN,NaN,1.010101,NaN,NaN,NaN
2,1915,10.1,10.000000,9.9,10.1,1.000000,NaN,NaN,NaN
3,1916,10.9,10.333333,10.0,10.9,7.920792,3.310298,1.0,7.920792
4,1917,12.8,11.266667,10.1,12.8,17.431193,8.783995,1.0,17.431193


<h2>Monthly CPI and Inflation Data</h2>

,Date,CPI,Inflation,RollingAvgCPI,RollingAvgCPI_low,RollingAvgCPI_high,RollingAvgInflation,RollingAvgInflation_low,RollingAvgInflation_high
0,01/01/1913,9.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,02/01/1913,9.8,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
220,03/01/1913,9.8,0.000000,9.800000,9.8,9.8,NaN,NaN,NaN
330,04/01/1913,9.8,0.000000,9.800000,9.8,9.8,0.000000,0.000000,0.0
440,05/01/1913,9.7,-1.020408,9.766667,9.7,9.8,-0.340136,-1.020408,0.0


## Plot CPI and inflation data with rolling avg

In [64]:
'''
CPI Yearly
'''

fig = go.Figure()

y = cpi_yearly['CPI']
x = cpi_yearly['Date']
y_roll = cpi_yearly['RollingAvgCPI']
y_rollmin = cpi_yearly['RollingAvgCPI_low']
y_rollmax = cpi_yearly['RollingAvgCPI_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Yearly CPI'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window} Year Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window} Year Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window} Year Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
display(Markdown('<h2>Yearly CPI Data</h2>'))
fig.show(config=scatter_config)

'''
CPI Monthly
'''

fig = go.Figure()

y = cpi_monthly['CPI']
x = cpi_monthly['Date']
y_roll = cpi_monthly['RollingAvgCPI']
y_rollmin = cpi_monthly['RollingAvgCPI_low']
y_rollmax = cpi_monthly['RollingAvgCPI_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Monthly CPI'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window} Month Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window} Month Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window} Month Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
display(Markdown('<h2>Monthly CPI Data</h2>'))
fig.show(config=scatter_config)

'''
Inflation Yearly
'''

fig = go.Figure()

y = cpi_yearly['Inflation']
x = cpi_yearly['Date']
y_roll = cpi_yearly['RollingAvgInflation']
y_rollmin = cpi_yearly['RollingAvgInflation_low']
y_rollmax = cpi_yearly['RollingAvgInflation_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Yearly Inflation'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window} Year Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window} Year Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window} Year Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
display(Markdown('<h2>Yearly Inflation Data</h2>'))
fig.show(config=scatter_config)


'''
Inflation Monthly
'''

fig = go.Figure()

y = cpi_monthly['Inflation']
x = cpi_monthly['Date']
y_roll = cpi_monthly['RollingAvgInflation']
y_rollmin = cpi_monthly['RollingAvgInflation_low']
y_rollmax = cpi_monthly['RollingAvgInflation_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Monthly Inflation'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window} Month Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window} Month Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window} Month Rolling High'))

#fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
display(Markdown('<h2>Monthly Inflation Data</h2>'))

fig.show(config=scatter_config)

<h2>Yearly CPI Data</h2>

<h2>Monthly CPI Data</h2>

<h2>Yearly Inflation Data</h2>

<h2>Monthly Inflation Data</h2>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c5ceaca9-e365-4132-a867-b543164c96b0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>